In [1]:
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import scipy.misc
import os
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
cascPath = "haarcascade_frontalface_default.xml"
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)


In [164]:
## read dataset1
full_face = []
full_labels = []
all_images = os.listdir('yalefaces')
for j in range(1,16):    # there are 15 different people in the dataset
    img_list = [filename for filename in all_images if filename.startswith('subject%.2d'%j)]
#     img_list = glob.glob('yalefaces/subject%.2d*'%j)
    face1 = np.zeros((len(img_list),32,32))
    for k,img in enumerate(img_list):
        I = scipy.misc.imread(os.path.join('yalefaces',img))
        I = cv2.fastNlMeansDenoising(I, None, 9, 13)
        I = cv2.GaussianBlur(I, (5,5), 0)
        I = cv2.equalizeHist(I)
        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        try : 
            x,y,h,w = faces[0]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])     
        except : 
            x,y,h,w=[0,0,479,639]
            face1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])   
        
    labels = (j-1)*np.ones((11,))
    full_face.append(face1)
    full_labels.append(labels)
full_face = np.stack(full_face)    # contains the images of all faces
full_labels = np.stack(full_labels)   # contains the ID of all the faces


In [165]:
full_face = full_face/255 #normalise data

In [166]:
full_face.shape

(15, 11, 32, 32)

In [167]:
full_labels.shape

(15, 11)

In [125]:
#similar 1 dissimilar 0
# 2 images of the same person
pairs = []
labels = []
similarity = []
for i in range(0,15):
    for j in range(50):
        a = np.random.randint(0,11,2)
        labels.append([i,i])
        pairs.append(a)
        similarity.append(1)

In [176]:
#similar 1 dissimilar 0
# 2 images of same and different people
similarity=[]
labels = []
similarity = []
sim=0
nonsim=0
for i in range(0,10000):
        b = np.random.randint(0,15,2)
        a = np.random.randint(0,11,2)
        
        
        if(b[0]==b[1]):
            similarity.append(1)
            labels.append(b)
            pairs.append(a)
            sim+=1
        else:
            if nonsim>sim : 
                continue
            else : 
                similarity.append(0)
                labels.append(b)
                pairs.append(a)
                nonsim+=1

In [177]:
print(len(similarity),sum(similarity))

1339 669


In [178]:
trainX = np.zeros((len(similarity),32,32))

In [179]:
a

array([9, 9])

In [180]:
face1 = full_face[a[0],b[0],:,:]

In [181]:
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances

In [182]:
X_train = np.zeros((len(similarity),32*32))
for i in range(len(similarity)):
    a = labels[i]
    b = pairs[i]
    face1 = full_face[a[0],b[0],:,:]
    face2 = full_face[a[1],b[1],:,:]
    face = face1-face2
    face = euclidean_distances(face,face)
    trainX[i,:,:]=face
    X_train[i,:]=trainX[i,:,:].ravel()

In [183]:
X_train.shape

(1339, 1024)

In [184]:
xtrain1,xtest1,ytrain,ytest=train_test_split(X_train, similarity)

In [185]:
xtrain1.shape

(1004, 1024)

In [139]:
# if PCA
from sklearn.decomposition import PCA
pca = PCA(whiten=True)
pca.fit(xtrain1)
n_components = np.argmin(np.cumsum(pca.explained_variance_ratio_)<=0.99)
print(n_components)
xtrain=pca.transform(xtrain1)
xtrain=xtrain[:,:n_components]
xtest=pca.transform(xtest1)
xtest=xtest[:,:n_components]

85


In [65]:
# if LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda=LinearDiscriminantAnalysis()
lda.fit(xtrain1,ytrain)
xtrain=lda.transform(xtrain1)
xtest=lda.transform(xtest1)
print(xtrain.shape,xtest.shape)

(2437, 1) (813, 1)


In [186]:
# if nothing
xtrain=xtrain1
xtest=xtest1

In [187]:
from sklearn.metrics import f1_score

In [188]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5,5), random_state=1)
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(1), random_state=1)
clf.fit(xtrain,ytrain)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [189]:
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  0.9721115537848606
Test Accuracy :  0.8805970149253731
Train F1 Score :  0.9714867617107942
Test F1 Score :  0.8863636363636365


In [68]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C=50)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  0.9684037751333607
Test Accuracy :  0.9212792127921279
Train F1 Score :  0.9446441409058232
Test F1 Score :  0.8565022421524664


In [190]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=4,random_state=0)
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  0.8884462151394422
Test Accuracy :  0.8
Train F1 Score :  0.8808510638297872
Test F1 Score :  0.8023598820058997


In [191]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(criterion='entropy',max_depth=7)#,max_features='sqrt')
clf.fit(xtrain,ytrain)
print('Train Accuracy : ',clf.score(xtrain,ytrain))
print('Test Accuracy : ',clf.score(xtest,ytest))
ypred_train=clf.predict(xtrain)
ypred_test=clf.predict(xtest)
print('Train F1 Score : ',f1_score(y_pred=ypred_train,y_true=ytrain))
print('Test F1 Score : ',f1_score(y_pred=ypred_test,y_true=ytest))

Train Accuracy :  0.9342629482071713
Test Accuracy :  0.8208955223880597
Train F1 Score :  0.9313929313929313
Test F1 Score :  0.8245614035087718


In [192]:
import h5py
h5f = h5py.File('test.h5','r')
a = list(h5f.keys())
## read the dataset
full_face = []
full_labels = []
img_list=[h5f[i] for i in a]
tface1 = np.zeros((len(img_list),32,32))
for k,img in enumerate(img_list):
    I = np.array(img)#scipy.misc.imread(os.path.join('yalefaces',img))
    #print(type(I))
    I = cv2.fastNlMeansDenoising(I, None, 9, 13)
    I = cv2.GaussianBlur(I, (5,5), 0)
    I = cv2.equalizeHist(I)
    faces = faceCascade.detectMultiScale(
        I,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30))
    #print(k)
    #print(faces)
    try : 
        x,y,h,w = faces[0]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])
    except : 
        x,y,h,w=[0,0,111,91]
        tface1[k,...] = zoom(I[y:y+h,x:x+w],[32./h,32./w])

In [193]:
tface1.shape

(120, 32, 32)

In [194]:
tface1 = tface1/255

In [195]:
test_pairs = np.genfromtxt('image_pairs.csv', dtype=None, delimiter=',', names=True)

In [197]:
euc=[]

for i in range(3540):#(3540):
    x1 = np.zeros((32,32))
    x0 = np.zeros((32,32))
    ids = test_pairs[i][0]
    image1=test_pairs[i][1]
    image2=test_pairs[i][2]
    x0 = tface1[a.index(str(test_pairs[i][1])),:,:]
    x1 = tface1[a.index(str(test_pairs[i][2])),:,:]
    if(i%100==0):
        print(i)
    face = x0-x1
    face = euclidean_distances(face,face)
    face=face.ravel()
    inp = np.zeros((1,32*32))
    inp[0,:]=face
    #inp=pca.transform(inp)
    #inp=inp[:,:n_components]
    euc.append(clf.predict(inp))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500


In [201]:
#1 is similar; 0 is different
with open('dt2.csv','w') as f : 
    f.write('Id,Expected\n')
    for i in range(len(test_pairs)) : 
        ids = test_pairs[i][0]
        temp = euc[i][0]
        f.write(str(ids)+','+str(temp)+'\n')

In [198]:
euc = np.asarray(euc)

In [199]:
sum(clf.predict(X_train))

635

In [200]:
sum(euc)

array([216])